In [ ]:
import tensorcircuit as tc
from tensorcircuit.cloud import apis
from tensorcircuit.results import counts
from tensorcircuit.results.readout_mitigation import ReadoutMit
from tensorcircuit.compiler.qiskit_compiler import qiskit_compile
import networkx as nx
from mitiq import zne, ddd
from functools import partial
import numpy as np
import time

In [ ]:
# This block is the private token (All non-Tencent Quantum Lab users cannot run)
apis.set_token('v0kdNWjG7QoD8c46JIez8U.YgoFgbPLk4xWUW3bbvl2n8R1F62368dx90UvdtWD93tsxGaa0jyelIyP1vlHq53UqIz.knUzBQDIIEIeIfdnbs5.EhefhkK4PCmqkXnx1GyNHmb-04EBlbFIMbeIjJkx3')


In [ ]:
# This block is to call the 20 qubit device owned by Tencent Quantum Lab
mit = ReadoutMit("20xmon")
mit.cals_from_system(list(range(20)), shots=10000, method="local")

In [ ]:

def QAOAloss(graph, w, params,error_mit = True):
    nqubit = 20
    nlayers = 3

    graph, qlist = devicequbit(graph)

    # prime circuit
    c = tc.Circuit(nqubit)
    for i in qlist:
        c.h(i)

    for i in range(nlayers):    
        for e in range(len(graph)):
            c.cnot(graph[e][0],graph[e][1])
            c.rz(graph[e][1],theta=params[i,0]*w[e])
            c.cnot(graph[e][0],graph[e][1])

        for k in qlist:
            c.rx(k, theta=params[i,1]*2)
            
    t = apis.submit_task(circuit=c, shots=10000, device="simulator:tc")
    raw_count = t.results(blocked=True)
    ideal = sum([-counts.expectation(raw_count, z=[graph[e][0], graph[e][1]])*w[e] for e in range(len(graph))])

    d = apis.get_device("20xmon")
    c1, info = qiskit_compile(
        c,
        compiled_options={
            "basis_gates": ['h', 'rz', 'x', 'y', 'z', 'cz'],
            "optimization_level": 2,
        },
    )


    if error_mit == True:

        def execute(circuit):
            c = tc.Circuit.from_qiskit(circuit, 20)
            t = apis.submit_task(circuit=c, shots=10000, device=d,enable_qos_qubit_mapping=False,enable_qos_gate_decomposition=False)
            count =t.results(blocked=True)
            a = sum([-mit.expectation(count, z=[graph[e][0], graph[e][1]],**info)*w[e] for e in range(len(graph))])
            time.sleep(1.0)
            return a

        random_state = np.random.RandomState(0)
        noise_scaling_function = partial(
            zne.scaling.fold_gates_at_random,
            fidelities = {"single": 1.0},  
            random_state=random_state, 
        )
        factory = zne.inference.PolyFactory(scale_factors=[1, 3,  5],order = 2)
        

        qisc = c1.to_qiskit()
        result = zne.execute_with_zne(
            circuit=qisc, executor = execute, factory=factory, scale_noise = noise_scaling_function,num_to_average = 1
        )
  
    else:
        t = apis.submit_task(circuit=c1,shots=10000,device=d,enable_qos_qubit_mapping=False,enable_qos_gate_decomposition=False)
        raw_count = t.results(blocked=True)
        result = sum([-counts.expectation(raw_count, z=[graph[e][0], graph[e][1]])*w[e] for e in range(len(graph))])


    return  result, ideal


def devicequbit (graph):
    logical_physical_mapping={0:17,1:7,2:18,3:16,4:15}
    device_graph = []
    qlist = []
    for e in range(len(graph)):
        device_graph.append((logical_physical_mapping[graph[e][0]],logical_physical_mapping[graph[e][1]]))
        for i in range(2):
            if logical_physical_mapping[graph[e][i]] not in qlist:
                qlist.append(logical_physical_mapping[graph[e][i]])
     
    return device_graph, qlist



In [ ]:
# graph to be optimized
graph = [(0,1),(0,2),(0,3),(3,4)]
w = [1,-1,1,-1]
G=nx.Graph()
G.add_edges_from(graph)
nx.draw(G,with_labels=True)

In [ ]:
initial_X = np.random.uniform(low= 0, high= 1, size=[1, 4])
np.save('results/hardware/initialization/initial_X_trial0_p2.npy', initial_X)
raw_value, ideal_value = QAOAloss(graph, w, initial_X.reshape(2,2), error_mit = False)
mit_value, ideal_value = QAOAloss(graph, w, initial_X.reshape(2,2), error_mit = True)
print(initial_X)
print(raw_value, mit_value, ideal_value)

In [ ]:
def eval_objective(x, graph):
    """This is a helper function we use to unnormalize and evalaute a point"""
    a=np.array(x).reshape(2,2)
    raw_value, ideal_value = QAOAloss(graph, [1,-1,1,-1], a, error_mit = False)
    mit_value, ideal_value = QAOAloss(graph, [1,-1,1,-1], a, error_mit = True)
    return -raw_value, -mit_value, -ideal_value

In [ ]:
# DARBO settings
import torch
import odbo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float
batch_size, failure_tolerance = 1, 10
acqfn='ucb'
nlayers = 2
tr_length= [1.6]
switch, switch_counter = 'small', 0
mode = 'mit' # you could choose to optimize on raw, mitigated or ideal values

## Note TuRBO is working on only [0,1] parameter range, so we need to convert the true parameter spaces
## First point by BO is actually just a random selection 
X_new = []
for i in range(initial_X.shape[1]):
    if initial_X[0,i] <=0.5:
        X_new.append(initial_X[0,i]+0.5)
    else:
        X_new.append(initial_X[0,i]-0.5)
X_new = np.array(X_new).reshape(1, 2 * nlayers) #X_new = np.random.uniform(low= 0, high= 1, size=[1, 2 * nlayers]) if you want a random pick

paras = np.vstack([initial_X, X_new])
if switch == 'large':
    X_turbo = torch.tensor(paras/2/np.pi+0.5) # transform space from [-pi,pi] to [0,1]
    data = torch.tensor([eval_objective(x*2*np.pi-np.pi, graph) for x in X_turbo], dtype=dtype, device=device).unsqueeze(-1)
    Y_raw, Y_mit, ideal_value = data[:, 0], data[:, 1], data[:, 2]
if switch == 'small':
    X_turbo = torch.tensor(paras/np.pi+0.5) # transform space from [-pi/2,pi/2] to [0,1]
    data = torch.tensor([eval_objective(x*np.pi-np.pi/2, graph) for x in X_turbo], dtype=dtype, device=device).unsqueeze(-1)
    Y_raw, Y_mit, ideal_value  = data[:, 0], data[:, 1], data[:, 2]

if mode == 'raw':
    Y_turbo = Y_raw
elif mode == 'mit':
    Y_turbo = Y_mit
elif mode == 'ideal':
    Y_turbo = ideal_value    

# prepare TR state
state = odbo.turbo.TurboState(dim=X_turbo.shape[1], batch_size=batch_size, length=tr_length, n_trust_regions=len(tr_length), failure_tolerance = failure_tolerance)
state.best_value = Y_turbo.max()
bo_best = list(np.min(np.array(-Y_turbo), axis = 1))


In [ ]:
# DARBO Optimizer (we only show the 2 searching region case)
for i in range(98):
    if len(Y_turbo) > 100:
        break
    # check if we need to switch the searching parameter range.
    if switch_counter >=4:
        if switch == 'small':
            switch = 'large'
            X_turbo = X_turbo/2
        else:
            switch = 'small'
            X_turbo = X_turbo*2
        switch_counter = 0
    
    X_next, acq_value, ids = odbo.run_exp.turbo_design(state=state,X=X_turbo,Y=Y_turbo, n_trust_regions=len(tr_length), batch_size=batch_size, acqfn=acqfn, normalize=False, verbose=False)
    X_next = torch.reshape(X_next, [len(tr_length)*batch_size, 2*nlayers])
    if switch == 'small':
        X_next_paras = X_next*np.pi-np.pi/2      
    else:
        X_next_paras = X_next*2*np.pi-np.pi
    paras = np.vstack([paras, np.array(X_next_paras)])
    data_next = torch.tensor([eval_objective(x, graph) for x in X_next_paras], dtype=dtype, device=device)
    Y_raw_next, Y_mit_next, ideal_next = data_next[:, 0], data_next[:, 1], data_next[:, 2]

    if mode == 'raw':
        Y_next = Y_raw_next
    elif mode == 'mit':
        Y_next = Y_mit_next
    elif mode == 'ideal':
        Y_next = ideal_next
    
    if np.max(np.array(Y_next)) < np.max(np.array(Y_turbo)):
        switch_counter = switch_counter + 1
        
    # Update state
    state = odbo.turbo.update_state(state=state, Y_next=torch.reshape(Y_next, [len(tr_length), batch_size, 1]))
    X_turbo = torch.cat((X_turbo, X_next), dim=0)
    Y_turbo = torch.cat((Y_turbo, Y_next.unsqueeze(-1)), dim=0)
    Y_mit = torch.cat((Y_mit, Y_mit_next.unsqueeze(-1)), dim=0)
    Y_raw = torch.cat((Y_raw, Y_raw_next.unsqueeze(-1)), dim=0)
    ideal_value = torch.cat((ideal_value, ideal_next.unsqueeze(-1)), dim=0)
    
    # Check if this search is a failure or not. Fail then add a switch counter
    print(f"{i+1}) New loss: {-Y_next.item(): .4e} Best loss: {-state.best_value:.4e}, TR length: {state.length}")
    print(X_next_paras, -Y_raw[-1], -Y_mit[-1], -ideal_value[-1])
    bo_best.append(-Y_turbo.max())
    np.save('results/hardware/opt{}/X_{}_hardware_exp_DARBO_opt{}_100_trial0_p{}.npy'.format(mode, mode, mode, nlayers), paras)
    np.save('results/hardware/opt{}/Y_raw_hardware_exp_DARBO_opt{}_100_trial0_p{}.npy'.format(mode, mode, nlayers), -np.array(Y_raw))
    np.save('results/hardware/opt{}/Y_mit_hardware_exp_DARBO_opt{}_100_trial0_p{}.npy'.format(mode, mode, nlayers), -np.array(Y_mit))
    np.save('results/hardware/opt{}/Y_ideal_hardware_exp_DARBO_opt{}_100_trial0_p{}.npy'.format(mode, mode, nlayers), -np.array(ideal_value))
